In [1]:
!git clone https://github.com/Latortuga13/CSE151Diabetes.git

fatal: destination path 'CSE151Diabetes' already exists and is not an empty directory.


In [2]:

from re import A
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#accessing our data
data = pd.read_excel("CSE151Diabetes/Patients Data ( Used for Heart Disease Prediction ).xlsx")


In [3]:
# There is no missing data in our dataset
data.isnull().sum()

# dropping columns we discussed in the README.
data_clean = data.drop(columns=["PatientID", 'HeightInMeters', "WeightInKilograms", "State", "SmokerStatus", "ECigaretteUsage", "TetanusLast10Tdap"])
dSample = data_clean.sample(frac=0.01, random_state=42, axis=0)
dSample.info()

#cleaning our data
data_clean["AgeCategory"] = data["AgeCategory"].str.extract(r"(\d+)")
data_clean["AgeCategory"] = data_clean["AgeCategory"].astype(int)

data_clean = pd.get_dummies(data_clean, columns=['RaceEthnicityCategory'])

health_mapping = {
    "Excellent": 5,
    "Very good": 4,
    "Good": 3,
    "Fair": 2,
    "Poor": 1
}
data_clean["GeneralHealth"] = data_clean["GeneralHealth"].map(health_mapping)
data_clean["GeneralHealth"] = data_clean["GeneralHealth"].astype(int)

sex_mapping = {
    "Male": 1,
    "Female": 0
}

data_clean["Sex"] = data_clean["Sex"].map(sex_mapping)
data_clean["Sex"] = data_clean["Sex"].astype(int)

data_clean.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2376 entries, 39973 to 32459
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Sex                        2376 non-null   object 
 1   GeneralHealth              2376 non-null   object 
 2   AgeCategory                2376 non-null   object 
 3   BMI                        2376 non-null   float64
 4   HadHeartAttack             2376 non-null   int64  
 5   HadAngina                  2376 non-null   int64  
 6   HadStroke                  2376 non-null   int64  
 7   HadAsthma                  2376 non-null   int64  
 8   HadSkinCancer              2376 non-null   int64  
 9   HadCOPD                    2376 non-null   int64  
 10  HadDepressiveDisorder      2376 non-null   int64  
 11  HadKidneyDisease           2376 non-null   int64  
 12  HadArthritis               2376 non-null   int64  
 13  HadDiabetes                2376 non-null   objec

,Sex,GeneralHealth,AgeCategory,BMI,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,...,HIVTesting,FluVaxLast12,PneumoVaxEver,HighRiskLastYear,CovidPos,"RaceEthnicityCategory_Black only, Non-Hispanic",RaceEthnicityCategory_Hispanic,"RaceEthnicityCategory_Multiracial, Non-Hispanic","RaceEthnicityCategory_Other race only, Non-Hispanic","RaceEthnicityCategory_White only, Non-Hispanic"
0,0,2,75,32.099998,0,1,0,1,1,0,...,0,0,1,0,1,False,False,False,False,True
1,0,4,65,27.990000,0,0,0,0,0,0,...,0,1,1,0,0,False,False,False,False,True
2,1,5,60,22.530001,0,0,0,0,0,0,...,0,0,0,0,0,False,False,False,False,True
3,1,4,70,30.129999,0,0,0,0,0,0,...,0,1,1,0,0,False,False,False,False,True
4,0,3,50,27.760000,0,0,0,0,0,0,...,0,1,0,0,0,True,False,False,False,False


In [4]:
#Scaled the data to make it easier to compare the weights generated from our model for each feature afterwards.
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

age = pd.DataFrame(data_clean, columns=['AgeCategory'])
age = min_max_scaler.fit_transform(age)
data_clean['AgeCategory'] = age

bmi = pd.DataFrame(data_clean, columns=['BMI'])
bmi = min_max_scaler.fit_transform(bmi)
data_clean['BMI'] = bmi
data_clean.head()

,Sex,GeneralHealth,AgeCategory,BMI,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,...,HIVTesting,FluVaxLast12,PneumoVaxEver,HighRiskLastYear,CovidPos,"RaceEthnicityCategory_Black only, Non-Hispanic",RaceEthnicityCategory_Hispanic,"RaceEthnicityCategory_Multiracial, Non-Hispanic","RaceEthnicityCategory_Other race only, Non-Hispanic","RaceEthnicityCategory_White only, Non-Hispanic"
0,0,2,0.919355,0.234497,0,1,0,1,1,0,...,0,0,1,0,1,False,False,False,False,True
1,0,4,0.758065,0.186500,0,0,0,0,0,0,...,0,1,1,0,0,False,False,False,False,True
2,1,5,0.677419,0.122737,0,0,0,0,0,0,...,0,0,0,0,0,False,False,False,False,True
3,1,4,0.838710,0.211491,0,0,0,0,0,0,...,0,1,1,0,0,False,False,False,False,True
4,0,3,0.516129,0.183814,0,0,0,0,0,0,...,0,1,0,0,0,True,False,False,False,False


In [5]:
#Splitting data to run model
from sklearn.model_selection import train_test_split

X = data_clean.drop("HadDiabetes", axis=1)
Y = data_clean['HadDiabetes']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(190104, 31)
(47526, 31)
(190104,)
(47526,)


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearSVC(max_iter=1000000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                         precision    recall  f1-score   support

                                     No       0.84      0.99      0.91     39410
No, pre-diabetes or borderline diabetes       0.00      0.00      0.00      1068
                                    Yes       0.57      0.10      0.17      6667
Yes, but only during pregnancy (female)       0.00      0.00      0.00       381

                               accuracy                           0.83     47526
                              macro avg       0.35      0.27      0.27     47526
                           weighted avg       0.78      0.83      0.78     47526



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
#running of cleaned HadDiabetes Column along with a model using only the test data
data_clean = data_clean[data_clean["HadDiabetes"] != "Yes, but only during pregnancy (female)"]
data_clean = data_clean[data_clean["HadDiabetes"] != "No, pre-diabetes or borderline diabetes"]

X = data_clean.drop("HadDiabetes", axis=1)
Y = data_clean['HadDiabetes']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for c in [.1, .5, 1, 5, 10, 100]:
  model = LinearSVC(max_iter=1000000, C=c)
  model.fit(X_train_scaled, y_train)
  y_pred = model.predict(X_test_scaled)

  print(f"Using regularization coefficient c={c}: \n")
  print(classification_report(y_test, y_pred))



Using regularization coefficient c=0.1: 

              precision    recall  f1-score   support

          No       0.87      0.99      0.92     39450
         Yes       0.60      0.11      0.18      6654

    accuracy                           0.86     46104
   macro avg       0.73      0.55      0.55     46104
weighted avg       0.83      0.86      0.82     46104

Using regularization coefficient c=0.5: 

              precision    recall  f1-score   support

          No       0.87      0.99      0.92     39450
         Yes       0.60      0.11      0.18      6654

    accuracy                           0.86     46104
   macro avg       0.73      0.55      0.55     46104
weighted avg       0.83      0.86      0.82     46104

Using regularization coefficient c=1: 

              precision    recall  f1-score   support

          No       0.87      0.99      0.92     39450
         Yes       0.60      0.11      0.18      6654

    accuracy                           0.86     46104
   m

In [12]:
#running a model with oversampled Yes data to have a more balanced data set
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_resampled)
#X_test_scaled = scaler.transform(X_test)

for c in [.0001, .001, .01, 1, 10000]:
  model = LinearSVC(max_iter=1000000, C=c, penalty="l1")
  model.fit(X_resampled, y_resampled)
  y_pred = model.predict(X_test)
  y_pred2 = model.predict(X_train)


  print(f"Using regularization coefficient c={c}: \n")
  print('New set of reports')
  print(classification_report(y_test, y_pred, digits=5))
  print(classification_report(y_train, y_pred2, digits=5))

Using regularization coefficient c=0.0001: 

New set of reports
              precision    recall  f1-score   support

          No    0.93960   0.72524   0.81862     39489
         Yes    0.30556   0.72169   0.42934      6615

    accuracy                        0.72473     46104
   macro avg    0.62258   0.72347   0.62398     46104
weighted avg    0.84863   0.72473   0.76277     46104

              precision    recall  f1-score   support

          No    0.94012   0.72178   0.81661    157974
         Yes    0.30378   0.72530   0.42821     26440

    accuracy                        0.72229    184414
   macro avg    0.62195   0.72354   0.62241    184414
weighted avg    0.84888   0.72229   0.76092    184414

Using regularization coefficient c=0.001: 

New set of reports
              precision    recall  f1-score   support

          No    0.94629   0.73355   0.82645     39489
         Yes    0.32085   0.75147   0.44970      6615

    accuracy                        0.73612     46104
 

#Where does your model fit in the fitting graph? and What are the next models you are thinking of and why?

The predictive error between the test data and train data is still close (even given how many entries given in our dataset), and due to this we believe our model is underfitted. The next model we are thinking of is a decision tree model. They are easy to visualize and understand, and they'll help us figure out which features are most important to determine diabetes (the ultimate goal of the project). The trees also can handle many different types of data which our dataset has a lot of. We will still need to be careful with overfitting and generalization, so we will try various hyperparemeterizations and regularization coefficients.

In [16]:
from sklearn.metrics import confusion_matrix

model = LinearSVC(max_iter=1000000, C=0.01, penalty="l1")
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)



In [20]:
#6 bullet point on canvas
print("True Negatives: " + str(cm[0][0]))
print("False Positives: " + str(cm[0][1]))
print("False Negatives: " + str(cm[1][0]))
print("True Positives: " + str(cm[1][1]))

True Negatives: 28996
False Positives: 10493
False Negatives: 1635
True Positives: 4980
